#Author:Brian Erichsen Fagundes
#MSD CS 6017 - Summer - 2024
#Homework 2

Part 1: A/B Hypothesis testing

You conduct an A/B test for your website by showing A to some of your users, and B to others.

ad clicks	total views
A	500	1000
B	550	1000
It looks like B is better than A...

Perform a hypothesis test using the two-proportion z-value test See the "pooled" version from the table here statistic at both the 5% and 1% signficance levels.

Begin by formulating and writing down your null hypothesis, then compute the Z value for this trial, and its associated p-value.

Note: the Z value you compute is assumed to be distributed according to the standard normal distribution (mean = 0, variance = 1). We'll consider the "1-sided" p-value which meaures the probability of a sample having a Z value that is more extreme in only one tail (to get the 2-sided value, you'd just multiply by 2).

What can you conclude?

Note, to help you understand your results, you might want to tweak the number of clicks for b to see how that changes your results. For example, what do you expect to happen to your p value if b is clicked 560 or 570 times? This is JUST for understanding the results, and shouldn't bias your decision one way or another!

- Null hypotheses H0: The proportions of ad click for logos A and B are the same. H0 = pA = pB
- Alterntive hypotheses H1: The proportions of ad clicks for logos B is higher than for logo A. H1 = pB > pA

In [13]:
import math
from scipy.stats import norm
import statsmodels.api as sm

# Data
clicks_A = 500
views_A = 1000
clicks_B = 550
views_B = 1000

# p value for data a and data b
p_A = clicks_A / views_A
p_B = clicks_B / views_B

# In Binomial distribution mean = np and variance = p(1 - p)
# Var of sample proportion Var(p) = p(1 - p) / n
# Standard Error is the square root of the combined variance
# which simplified becomes sqrt of p_pool * ( 1 - p_pool) * (1 / popA + 1/ popB))
# z score is Z = (pB - pA) / SE

# pooled proportion
p_pooled = (clicks_A + clicks_B) / (views_A + views_B)

# Standard Error
SE = math.sqrt(p_pooled * (1 - p_pooled) * (1/views_A + 1/views_B))

Z = (p_B - p_A) / SE

# Cumulative Distribution Function - prob that  normally distributed
# random var is less or equal than given value

# One sided p-value // sees right of Z which is chance of
# observing an extreme value if H0 is true
p_value_one_sided = 1 - norm.cdf(Z)
p_value_two_sided = p_value_one_sided * 2

print(f"Pooled Proportion: {p_pooled}")
print(f"Stadandard Error: {SE}")
print(f"Z Value: {Z}")
print(f"One-sided p-value: {p_value_one_sided}")
print(f"Two-sided p-value: {p_value_two_sided}")

# Comfirming the results
count = [clicks_B, clicks_A]
nobs = [views_B, views_A]
z_stat, p_value = sm.stats.proportions_ztest(count, nobs, alternative='larger')
print(f"Z value (statsmodels): {z_stat}")
print(f"One-sided p-value (statsmodels): {p_value}")

# Significance levels
alpha_5 = 0.05
alpha_1 = 0.01

# Conclusions
if p_value < alpha_5:
    print("Reject the null hypothesis at the 5% significance level.")
else:
    print("Fail to reject the null hypothesis at the 5% significance level")



Pooled Proportion: 0.525
Stadandard Error: 0.022332711434127295
Z Value: 2.238868314198227
One-sided p-value: 0.012582242850159187
Two-sided p-value: 0.025164485700318373
Z value (statsmodels): 2.238868314198227
One-sided p-value (statsmodels): 0.0125822428501592
Reject the null hypothesis at the 5% significance level.


What can you conclude?

Reject the null hypothesis at the 5% significance level.


Part 2: Regression of real estate data
Adapted from datasciencecourse.net

For this problem, you will analyze SLC real estate data. The dataset contains multiple listing service (MLS) real estate transactions for houses sold in 2015-16 in zip code 84103 (SLC avenues neighborhod Google Map). We are primarily interested in regressing the SoldPrice on the house attributes (property size, house size, number of bedrooms, etc...).

Task 1: Import the data
Use the pandas.read_csv() function to import the dataset. The data is contained in two files: realEstate1.csv and realEstate2.csv. After you import these files separately, concatenate them into one big dataframe. This pandas dataframe will be used for data exploration and linear regression.

In [ ]:
import panda as pd

